<h1 align=center> Neural Networks - Logic Gates </h1>

<hr>
### Introduction

In this notebook I use **simple neural networks** to produce the output of different logic gates such as **AND, XNOR** or **IF**. The purpose of this notebook is to understand the basic structure of a neural network and particularly, what is the job of a neuron within it. The way the computations are being made resembles a **feed forward** algorithm, however, there is not **back propagation** as the networks are not being trained.

In [1]:
import numpy as np

from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

In [2]:
# uncomment this line in case you want to reproduce the results and plotly is not installed
#!pip install plotly

In [3]:
# helper function to print logic tables
def print_logic_table(X, outputs):
    '''given an input matrix X with x and y features, 
    and a list of outputs, prints the corresponding logic table.
    The format of outputs is [{"O": [y1, y2, y3, y4], "tag": OP_name}]'''

    header = "_ X _ Y __"
    row_structure = "| {} | {} ||"
    
    for output in outputs:
        header += " {} _".format(output["tag"])
        row_structure += "  {}  |"
        
    print(header)
    for i, row in enumerate(X):
        
        x, y = int(row[0]), int(row[1])
        output_row = [output["O"][i] for output in outputs]
        
        format_items = [x, y] + output_row
        
        print(row_structure.format(*format_items))

In [4]:
X_logic = np.array([[1,0,1,0], [1,1,0,0]]).T

outputs = [
    {"tag": "OR ", "O": [1, 1, 1, 0]},
    {"tag": "AND", "O": [1, 0, 0, 0]},
    {"tag": "IF ", "O": [1, 1, 0, 1]},
    {"tag": "XOR", "O": [0, 1, 1, 0]},
    {"tag": "XNOR", "O": [1, 0, 0, 1]}
]

print_logic_table(X_logic, outputs)

_ X _ Y __ OR  _ AND _ IF  _ XOR _ XNOR _
| 1 | 1 ||  1  |  1  |  1  |  0  |  1  |
| 0 | 1 ||  1  |  0  |  1  |  1  |  0  |
| 1 | 0 ||  1  |  0  |  0  |  1  |  0  |
| 0 | 0 ||  0  |  0  |  1  |  0  |  1  |


## Neurons and Perceptrons

In order to see what a neuron is doing, let's work with two weigths-one bias neurons. A neuron with two weights operates over an input with two features, such as our logic table (x, y). A Neuron is just a function that divides the space into two regions and is able to compute whether the input lies above or below the border. We can define this opperation as $g(\vec{x})$, where $\vec{x}$ is a single input. Then $g(\vec{x}) > 1$ if the input satisfies the Neuron equation, and $g(\vec{x}) < 1$ if it does not. 

$g(\vec{x}) = 1$ is the border between one input or another, and it represents the intersection of the input space with a hyperplane defined by the weights (in this case a normal plane with 3 dimensions). With this in mind we can find the line equation that defines the neuron:

$$
g(\vec{x}) = 1 = \vec{w} · \vec{x} + bias
= \begin{bmatrix} w_1 w_2\end{bmatrix}  \begin{bmatrix} x_{1} \\ x_{2} \end{bmatrix} + bias = w_1 x_1 + w_2 x_2 + bias 
$$

$$ x_2 = -\frac{w_1}{w_2} x_1 - \frac{bias}{w_2} $$

In [23]:
def draw_line(x, weights, bias):
    '''returns line eq based on weights and bias'''
    m = -weights[0] / weights[1]
    b = -bias / weights[1]
    
    y = m*x + b
    
    return y

In [6]:
def draw_neuron(neurons, clues=True, tags=None, title=None):
    
    try:
        _ = neurons[0][0]
    except:
        raise ValueError("Please pass a list of neurons")
        
    if tags is None:
        tags = ["" for _ in neurons]
        
    if title is None:
        title = ""
    
    neuron_lines = []
    
    x = np.linspace(-0.5, 1.5)
    
    for i, neuron in enumerate(neurons):
        
        if len(neuron) != 3:
            raise ValueError("only draw neurons of two dimensions (+ bias)")
    
        weights, bias = neuron[1:], neuron[0]
        y = draw_line(x, weights, bias)
        
        neuron_lines.append({"x": x, "y": y, "name": "Neuron {}".format(tags[i])})
    
    annotations = []
    if clues:
        neuron_lines.extend([{"x": [x_i], "y": [y_i], 
                              "marker": dict(symbol="square", size=10),
                              "name": "[{},{}]".format(x_i, y_i)} 
                             for x_i, y_i in zip([1,0,1,0], [1,1,0,0])]) 
        
        annotations = [dict(x=x_i+0.05, y=y_i+0.1, text="({}, {})".format(x_i, y_i),
                           showarrow=False)
                      for x_i, y_i in zip([1,0,1,0], [1,1,0,0])]
    
    
    layout = go.Layout(title=title, xaxis=dict(title="x1"), yaxis=dict(title="x2"),
                      annotations=annotations)
    fig = go.Figure(data=neuron_lines, layout=layout)
    iplot(fig)

In [7]:
draw_neuron([[-3, 2, 2], [-1, 2, 2], [1, -2, 2]], 
            tags=["AND", "OR", "IF"], title="Space Boundary from AND, OR and IF neurons")

### Neurons as perceptrons

In order to use a perceptron as a classifier, we need a function that returns 1 or 0 given a pair of inputs $x_1$ and $x_2 = g(x)$.

In [8]:
def step_function(x):
    return 1 if x >= 1 else 0
    
x = np.linspace(-0.5, 1.5, 1000)
data = [{"x": x, "y": [step_function(x_i) for x_i in x], "name": "step function"}]
layout = go.Layout(title="Step Function", xaxis=dict(title="x"), yaxis=dict(title="f(x)"))
fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [9]:
def perceptron(weights, bias, x):
    '''returns 1 if the perceptron gets triggered'''
    return step_function(np.dot(weights, x) + bias)

We can use the perceptrons as logic gates providing the following inputs (the same used to draw the lines above) 

In [10]:
perceptrons = [
    {"tag": "OR", "weights": [2, 2], "bias": -1},
    {"tag": "AND", "weights": [2, 2], "bias": -3},
    {"tag": "IF", "weights": [-2, 2], "bias": 1},
]

p_outputs = []

for p in perceptrons:
    output = {"tag": p["tag"], "O": [perceptron(p["weights"], p["bias"], [x, y]) 
                                     for x, y in zip([1,0,1,0],[1,1,0,0])]}
    p_outputs.append(output)

print_logic_table(X_logic, p_outputs)

_ X _ Y __ OR _ AND _ IF _
| 1 | 1 ||  1  |  1  |  1  |
| 0 | 1 ||  1  |  0  |  1  |
| 1 | 0 ||  1  |  0  |  0  |
| 0 | 0 ||  0  |  0  |  1  |


## Neural Networks: Combining neurons

With a single neuron we can obtain any of the above operations that can be **linearly separated**, such as **AND**, **IF**, or **OR**. However, if we want to separate [1,0] and [0,1] from [0,0] and [1,1] (which is a **XOR** or **XNOR** operation) we need a **non linear fuction**. To achieve this we can use the neurons **AND** and **OR** together.

Now we need a more sophisticated activation function, so we can use the sigmoid function. We will need the derivative of the activation function later on, so we can compute it now for completeness:

$f(x)=\frac{1}{1+e^{-x}}$
<br><br>
$f'(x)=f(x)·(1 - f(x))$

In [11]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_prime(x):
    return sigmoid(x)*(1 - sigmoid(x))

x = np.linspace(-10,10)
data = [{"x":x, "y": sigmoid(x), "name":"f(x): sigmoid"}, 
        {"x":x, "y": sigmoid_prime(x), "name": "f(x): sigmoid prime"}]
layout = go.Layout(title="Sigmoid Function", xaxis=dict(title="x"), yaxis=dict(title="f(x)"))
fig = go.Figure(data=data, layout=layout)
iplot(fig)

### Feed Forward


In order to ease the calculations, let's create a helper function to add a bias column to a given matrix

In [12]:
def add_bias(X):
    ''' add a column of ones to a 2d array'''
    l = X.shape[0]
    
    if X.shape == (l,):
        X = X.reshape(l,1)
    
    bias = np.ones(l).reshape(l, 1)
    return np.append(bias, X, axis=1)

With this we can compute the Feed Forward algorithm. Notes are provided as coments.

In [13]:
def forward(X, W1, W2, f1, bias=True):
    
    '''
    If we use bias we need an extra column in the inputs (x_0 = 1)
    '''
    if bias:
        X = add_bias(X) 
    
    '''
    Each neuron in the hidden layer is a column vector in W1
    z2 is the output of each of the neurons when fed with all the inputs:
        z2_1 = x_1 * W1_11 + x_2 * W1_12 
    where z2_1 is the output of the first neuron
    and W1_ij is the j-th weight of the i-th neuron in the first layer
    '''
    z2 = np.dot(X, W1) 
    
    '''
    After the output is computed, it is necessary to activate it,
    in this case, using the sigmoid function
    '''
    a2 = f1(z2)
    
    '''
    If we are using bias, we will need an extra column in a2, 
    the same way we needed it in X.
    a2 is going to be the input for the next layer 
    (the output laye rin this case)
    '''
    if bias:
        a2 = add_bias(a2)
    
    '''
    we repeat the process: produce a new output and activate it
    '''
    z3 = np.dot(a2, W2)
    a3 = f1(z3)
    
    '''
    in this case, a3 is already the output of the network
    '''
    
    Y = a3
    return Y

In [14]:
X_logic

array([[1, 1],
       [0, 1],
       [1, 0],
       [0, 0]])

### Logic Networks

Now let's see how we can create simple networks that will behave as logic gates when passeed to the feed forward algorithm

#### AND network

In [15]:
neuron_and = [-30,20,20]
draw_neuron([neuron_and], tags=["AND"], title="AND Neuron")

Let's put the neuron in a column:

In [16]:
W1_and = np.array([neuron_and]).T
print("W1 and:")
print(W1_and)

W1 and:
[[-30]
 [ 20]
 [ 20]]


For this single neuron layer we will need an output layer with a neuron that accepts only one input (plus bias). We can use a simple "yes" neuron.

In [17]:
W2_and = np.array([[-10,20]]).T
print("\nW2 and:")
print(W2_and)


W2 and:
[[-10]
 [ 20]]


In [18]:
fwd_and = forward(X_logic, W1_and, W2_and, sigmoid)
fwd_and

array([[  9.99954561e-01],
       [  4.54391049e-05],
       [  4.54391049e-05],
       [  4.53978687e-05]])

In [19]:
and_outputs = [
    {"tag":" AND (FWD) ", "O": ["{:.3E}".format(y) for y in fwd_and.flatten()]},
    {"tag": "AND", "O": [1,0,0,0]}
]

print_logic_table(X_logic, and_outputs)

_ X _ Y __  AND (FWD)  _ AND _
| 1 | 1 ||  1.000E+00  |  1  |
| 0 | 1 ||  4.544E-05  |  0  |
| 1 | 0 ||  4.544E-05  |  0  |
| 0 | 0 ||  4.540E-05  |  0  |


#### OR network

In [28]:
neuron_or = [-10, 20, 20]
draw_neuron([neuron_or], tags=["OR"], title="OR Neuron")

W1_or = np.array([neuron_or]).T
print("W1 or:")
print(W1_or)

W2_or = W2_and
print("\nW2 or:")
print(W2_or)

print("\n\n")

fwd_or = forward(X_logic, W1_or, W2_or, sigmoid)

or_outputs = [
    {"tag":" OR (FWD)  ", "O": ["{:.3E}".format(y) for y in fwd_or.flatten()]},
    {"tag": "OR ", "O": [1,1,1,0]}
]

print_logic_table(X_logic, or_outputs)

W1 or:
[[-10]
 [ 20]
 [ 20]]

W2 or:
[[-10]
 [ 20]]



_ X _ Y __  OR (FWD)   _ OR  _
| 1 | 1 ||  1.000E+00  |  1  |
| 0 | 1 ||  1.000E+00  |  1  |
| 1 | 0 ||  1.000E+00  |  1  |
| 0 | 0 ||  4.544E-05  |  0  |


#### NOR Network

NOR neuron resembles the OR neuron in the sense that they both delimitate the space in the same regions (their lines are identical). However, each of them gets triggered in the correspondent two subregions (they are complementary).

In [29]:
neuron_nor = [10, -20, -20]
draw_neuron([neuron_nor], tags=["NOR"], title="NOR Neuron")

W1_nor = np.array([neuron_nor]).T
print("W1 nor:")
print(W1_nor)

W2_nor = W2_and
print("\nW2 nor:")
print(W2_nor)

print("\n\n")

fwd_nor = forward(X_logic, W1_nor, W2_nor, sigmoid)

nor_outputs = [
    {"tag":" NOR (FWD)  ", "O": ["{:.3E}".format(y) for y in fwd_nor.flatten()]},
    {"tag": "NOR ", "O": [0,0,0,1]}
]

print_logic_table(X_logic, nor_outputs)

W1 nor:
[[ 10]
 [-20]
 [-20]]

W2 nor:
[[-10]
 [ 20]]



_ X _ Y __  NOR (FWD)   _ NOR  _
| 1 | 1 ||  4.540E-05  |  0  |
| 0 | 1 ||  4.544E-05  |  0  |
| 1 | 0 ||  4.544E-05  |  0  |
| 0 | 0 ||  1.000E+00  |  1  |


### Combined Networks

#### XNOR Network

For the XNOR network we will need to use two neurons in the first layer, and then a more complex neuron in the second layer (and not just the "yes" neuron).

Specifically, XNOR will be a combination of AND, NOR and OR neurons.

In [32]:
draw_neuron([neuron_and, neuron_or, neuron_nor], tags=["AND", "OR", "NOR"], title="AND, OR and NOR Neurons")

W1_xnor = np.array([neuron_and, neuron_nor]).T
print("W1 xnor:")
print(W1_xnor)

W2_xnor = np.array([neuron_or]).T
print("\nW2 xnor:")
print(W2_xnor)

print("\n\n")

fwd_xnor = forward(X_logic, W1_xnor, W2_xnor, sigmoid)

xnor_outputs = [
    {"tag":"XNOR (FWD)", "O": ["{:.3E}".format(y) for y in fwd_xnor.flatten()]},
    {"tag": "XNOR ", "O": [1,0,0,1]}
]

print_logic_table(X_logic, xnor_outputs)

W1 xnor:
[[-30  10]
 [ 20 -20]
 [ 20 -20]]

W2 xnor:
[[-10]
 [ 20]
 [ 20]]



_ X _ Y __ XNOR (FWD) _ XNOR  _
| 1 | 1 ||  1.000E+00  |  1  |
| 0 | 1 ||  4.548E-05  |  0  |
| 1 | 0 ||  4.548E-05  |  0  |
| 0 | 0 ||  1.000E+00  |  1  |


<br><br><br><br><br><br><br><br><br>